# Microsoft Copilot - Graph Connector for Formula 1 World Championship (1950 - 2023)

## Initialization code

In the following section we load all the needed libraries and types.
In order to run this sample you need to import the following Python packages:

- pip install python-dotenv
- pip install azure-identity
- pip install msgraph-sdk
- pip install requests


In [1]:
import pandas as pd
import requests
from datetime import datetime
import unicodedata
from typing import Generator
from urllib.parse import urljoin
import json
from dotenv import load_dotenv
import os
from azure.identity.aio import ClientSecretCredential
from msgraph import GraphServiceClient
from kiota_abstractions.api_error import APIError
from msgraph.generated.models.external_connectors.activity_settings import ActivitySettings
from msgraph.generated.models.external_connectors.display_template import DisplayTemplate
from msgraph.generated.models.external_connectors.external_connection import ExternalConnection
from msgraph.generated.models.external_connectors.item_id_resolver import ItemIdResolver
from msgraph.generated.models.external_connectors.search_settings import SearchSettings
from msgraph.generated.models.external_connectors.url_match_info import UrlMatchInfo
from msgraph.generated.models.external_connectors.schema import Schema
from msgraph.generated.models.external_connectors.property_ import Property_
from msgraph.generated.models.external_connectors.property_type import PropertyType
from msgraph.generated.models.external_connectors.label import Label
from msgraph.generated.models.json import Json

from msgraph.generated.models.external_connectors.access_type import AccessType
from msgraph.generated.models.external_connectors.acl import Acl
from msgraph.generated.models.external_connectors.acl_type import AclType
from msgraph.generated.models.external_connectors.external_activity import (
    ExternalActivity,
)
from msgraph.generated.models.external_connectors.external_activity_type import (
    ExternalActivityType,
)
from msgraph.generated.models.external_connectors.external_item import ExternalItem
from msgraph.generated.models.external_connectors.external_item_content import (
    ExternalItemContent,
)
from msgraph.generated.models.external_connectors.external_item_content_type import (
    ExternalItemContentType,
)
from msgraph.generated.models.external_connectors.identity import Identity
from msgraph.generated.models.external_connectors.identity_type import IdentityType
from msgraph.generated.models.external_connectors.properties import Properties

In [2]:
load_dotenv()

True

## Drivers data load and preparation

Here we load the CSV file with data about all the F1 drivers from 1950 to 2023 using Pandas and we show few information about the data

In [3]:
# Set the CSV file name
file_path = "data/drivers.csv"
 
# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Convert the dob field into a date field
df['dob'] = pd.to_datetime(df['dob'])

# Display the first few rows of the DataFrame
print(df.head())

   driverId   driverRef number code  forename     surname        dob  \
0         1    hamilton     44  HAM     Lewis    Hamilton 1985-01-07   
1         2    heidfeld     \N  HEI      Nick    Heidfeld 1977-05-10   
2         3     rosberg      6  ROS      Nico     Rosberg 1985-06-27   
3         4      alonso     14  ALO  Fernando      Alonso 1981-07-29   
4         5  kovalainen     \N  KOV    Heikki  Kovalainen 1981-10-19   

  nationality                                             url  
0     British     http://en.wikipedia.org/wiki/Lewis_Hamilton  
1      German      http://en.wikipedia.org/wiki/Nick_Heidfeld  
2      German       http://en.wikipedia.org/wiki/Nico_Rosberg  
3     Spanish    http://en.wikipedia.org/wiki/Fernando_Alonso  
4     Finnish  http://en.wikipedia.org/wiki/Heikki_Kovalainen  


Define helper functions to process data

In [4]:
def convert_to_iso8601(date_str):
    # Parse the date string to datetime object
    dt = datetime.strptime(date_str, '%Y-%m-%d')

    # Convert to ISO 8601 format
    iso8601_format = dt.isoformat() + "Z"  # Appending 'Z' to indicate UTC
    return iso8601_format

In [5]:
def read_driver_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        web_page_content = response.text
        return web_page_content
    else:
        print(f"Failed to fetch the web page: {url}")

In [6]:
def replace_special_characters(text):
    # Define a mapping of special characters to their replacements
    replacements = {
        'à': 'a',
        'ä': 'a',
        'á': 'a',
        'Ã': 'A',
        'Á': 'A',
        'è': 'e',
        'é': 'e',
        'É': 'E',
        'ì': 'i',
        'í': 'i',
        'ò': 'o',
        'ö': 'o',
        'ô': 'o',
        'ó': 'o',
        'ø': 'o',
        'ù': 'u',
        'ú': 'u',
        'ü': 'u',
        'ç': 'c',
        'š': 's',
        'ñ': 'n',
        '.': '',
        ' ': '',
        '\'': ''
    }

    # Iterate over the mapping and replace each special character with its replacement
    for special_char, replacement in replacements.items():
        text = text.replace(special_char, replacement)
    
    return text

## Connect to Microsoft Graph

In [7]:
credential = ClientSecretCredential(
    os.getenv("TENANT_ID"),
    os.getenv("CLIENT_ID"),
    os.getenv("CLIENT_SECRET")
)
scopes = ['https://graph.microsoft.com/.default']
client = GraphServiceClient(credentials=credential, scopes=scopes)

In [8]:
# Get the User ID needed as PerformedBy Identity
user_id = os.getenv("USER_ID")

## Define the Connector Schema

Define the connection

In [9]:
external_connection = ExternalConnection(
    id="paolopiaf1data",
    name="Formula 1 World Championship Drivers (born after 1975)",
    description="Statistical data about Formula 1 World Championship Drivers (born after 1975)",
    activity_settings=ActivitySettings(
        url_to_item_resolvers=[
            ItemIdResolver(
                odata_type="#microsoft.graph.externalConnectors.itemIdResolver",
                priority=1,
                item_id="{driverid}",
                url_match_info=UrlMatchInfo(
                    base_urls=[
                        "http://en.wikipedia.org/wiki"
                    ],
                    url_pattern="/(?<driverid>[a-z_A-Z]+)"
                )
            )
        ]
    ),
    search_settings=SearchSettings(
      search_result_templates=[
        DisplayTemplate(
            id="paolopiaf1data",
            priority=1
        )
      ]
    )
)

Define the schema

In [10]:
schema = Schema(
  base_type="microsoft.graph.externalItem",
  properties=[
    Property_(
        name="title",
        type=PropertyType.String,
        is_queryable=True,
        is_searchable=True,
        is_retrievable=True,
        labels=[
            Label.Title
        ]
    ),
    Property_(
        name="code",
        type=PropertyType.String,
        is_queryable=True,
        is_searchable=True,
        is_retrievable=True
    ),
    Property_(
        name="number",
        type=PropertyType.Int64,
        is_queryable=True,
        is_retrievable=True
    ),
    Property_(
        name="nationality",
        type=PropertyType.String,
        is_queryable=True,
        is_searchable=True,
        is_retrievable=True
    ),
    Property_(
        name="url",
        type=PropertyType.String,
        is_retrievable=True,
        labels=[
            Label.Url
        ]
    ),
    Property_(
        name="birthdate",
        type=PropertyType.DateTime,
        is_queryable=True,
        is_retrievable=True,
        is_refinable=True
    ),
    Property_(
        name="reference",
        type=PropertyType.String,
        is_queryable=True,
        is_retrievable=True,
        is_refinable=True
    ),
    Property_(
        name="driverid",
        type=PropertyType.Int64,
        is_retrievable=True
    )
  ]
)

Create the connection and the schema with Microsoft Graph

In [11]:
async def _create_connection():
    print("Creating connection...")
    with open("driverLayout.json", "r", encoding='utf-8') as file:
        adaptive_card = file.read()
        layout = json.loads(adaptive_card)

        assert external_connection.search_settings is not None
        assert external_connection.search_settings.search_result_templates is not None

        external_connection.search_settings.search_result_templates[0].layout = Json(
            additional_data=layout
        )

    try:
        # Try to see if the connection already exists and if that is the case, update it
        await client.external.connections.by_external_connection_id(external_connection.id).patch(external_connection)
    except Exception as e:
        # If it does not exist, let's create it
        await client.external.connections.post(external_connection)

    print("CONNECTION CREATED/UPDATE")

In [12]:
async def _create_schema():
    print("Creating schema...")

    assert external_connection.id is not None
    await client.external.connections.by_external_connection_id(external_connection.id).schema.patch(schema)
    print("DONE")

In [13]:
async def create_connection():
    await _create_connection()
    await _create_schema()

In [14]:
await create_connection()

Creating connection...
CONNECTION CREATED/UPDATE
Creating schema...
DONE


## Ingest the actual data (transform and load phase)

In [15]:
def _transform(content) -> Generator[ExternalItem, None, None]:
    currentDate = datetime.now().isoformat(timespec='seconds') + "Z"

    for index, driver in content.iterrows():
        print(driver["surname"])
        driverId = replace_special_characters(f"{driver["forename"]}_{driver["surname"]}")
        driverTitle = f"{driver["forename"]} {driver["surname"]}"
        driverNumber = driver["number"]
        driverBirthDate = (driver["dob"]).isoformat(timespec='seconds') + "Z"
        driverBirthDateShort = driver["dob"].strftime("%Y-%m-%d")
        driverContent = f"<div><h2>{driverTitle}</h2><p><b>{driverTitle}</b> is a {driver["nationality"]} F1 driver who was born on {driverBirthDateShort}.</p><p>Official number is {driverNumber} and official code is {driver["code"]}.</p><p>Additional details can be found at the following URL: <a href='{driver["url"]}'>{driver["url"]}</a></p></div>"
        # driverContent = read_driver_html(driver["url"])
        yield ExternalItem(
            id=driverId,
            properties=Properties(
                additional_data={
                    "title": driverTitle,
                    "code": driver["code"] if driverNumber != "\\N" else "",
                    "number": int(driverNumber) if driverNumber != "\\N" else 0,
                    "nationality": driver["nationality"],
                    "url": driver["url"],
                    "birthdate": driverBirthDate,
                    "reference": driver["driverRef"],
                    "driverid": driver["driverId"]
                }
            ),
            content=ExternalItemContent(
                type=ExternalItemContentType.Html, value=driverContent
            ),
            acl=[
                Acl(
                    type=AclType.Everyone,
                    value="everyone",
                    access_type=AccessType.Grant,
                )
            ],
            activities=[
                ExternalActivity(
                    odata_type="#microsoft.graph.externalConnectors.externalActivity",
                    type=ExternalActivityType.Created,
                    start_date_time=currentDate,
                    performed_by=Identity(type=IdentityType.User, id=user_id),
                )
            ],
        )

In [16]:
async def _load(content: Generator[ExternalItem, None, None]):
    counter = 0
    for doc in content:
        try:
            counter += 1
            print(f"{counter} - Loading {doc.id} ...")
            assert external_connection.id is not None
            assert doc.id is not None

            await client.external.connections.by_external_connection_id(
                external_connection.id
            ).items.by_external_item_id(doc.id).put(doc)
            print("DONE loading")
        except Exception as e:
            print(f"Failed to load {doc.id}. Error: {e}")
            return

In [17]:
# Transform only drivers who were born after 1975
transformed = _transform(df[df['dob'] >= '1975-01-01'])

In [19]:
result = await _load(transformed)

Heidfeld
1 - Loading Nick_Heidfeld ...
DONE loading
Rosberg
2 - Loading Nico_Rosberg ...
DONE loading
Alonso
3 - Loading Fernando_Alonso ...
DONE loading
Kovalainen
4 - Loading Heikki_Kovalainen ...
DONE loading
Nakajima
5 - Loading Kazuki_Nakajima ...
DONE loading
Bourdais
6 - Loading Sebastien_Bourdais ...
DONE loading
Räikkönen
7 - Loading Kimi_Raikkonen ...
DONE loading
Kubica
8 - Loading Robert_Kubica ...
DONE loading
Glock
9 - Loading Timo_Glock ...
DONE loading
Sato
10 - Loading Takuma_Sato ...
DONE loading
Piquet Jr.
11 - Loading Nelson_PiquetJr ...
DONE loading
Massa
12 - Loading Felipe_Massa ...
DONE loading
Sutil
13 - Loading Adrian_Sutil ...
DONE loading
Webber
14 - Loading Mark_Webber ...
DONE loading
Button
15 - Loading Jenson_Button ...
DONE loading
Davidson
16 - Loading Anthony_Davidson ...
DONE loading
Vettel
17 - Loading Sebastian_Vettel ...
DONE loading
Schumacher
18 - Loading Ralf_Schumacher ...
DONE loading
Liuzzi
19 - Loading Vitantonio_Liuzzi ...
DONE loading
Spe